In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datathonai/train.csv
/kaggle/input/datathonai/test.csv
/kaggle/input/datathonai/test/image_4378.jpg
/kaggle/input/datathonai/test/image_4722.jpg
/kaggle/input/datathonai/test/image_4432.jpg
/kaggle/input/datathonai/test/image_4776.jpg
/kaggle/input/datathonai/test/image_4710.jpg
/kaggle/input/datathonai/test/image_5161.jpg
/kaggle/input/datathonai/test/image_4953.jpg
/kaggle/input/datathonai/test/image_4700.jpg
/kaggle/input/datathonai/test/image_5008.jpg
/kaggle/input/datathonai/test/image_4735.jpg
/kaggle/input/datathonai/test/image_4752.jpg
/kaggle/input/datathonai/test/image_4634.jpg
/kaggle/input/datathonai/test/image_5085.jpg
/kaggle/input/datathonai/test/image_4653.jpg
/kaggle/input/datathonai/test/image_5025.jpg
/kaggle/input/datathonai/test/image_5091.jpg
/kaggle/input/datathonai/test/image_4433.jpg
/kaggle/input/datathonai/test/image_5050.jpg
/kaggle/input/datathonai/test/image_4683.jpg
/kaggle/input/datathonai/test/image_4505.jpg
/kaggle/input/datathonai/test/i

In [2]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader, random_split

from torchvision.transforms import (
    Compose,
    Resize,
    RandomHorizontalFlip,
    RandomRotation,
    RandomResizedCrop,
    ToTensor,
    Normalize
)

from transformers import AutoModelForImageClassification, AutoImageProcessor
import matplotlib.pyplot as plt
from tqdm import tqdm
import math


#############################
# 1. Normalizasyon Değerleri
#############################
LAT_MEAN = 41.10
LAT_STD = 0.002358
LON_MEAN = 29.02
LON_STD = 0.004205

BATCH_SIZE = 32
NUM_EPOCHS = 10
LEARNING_RATE = 5e-3


#############################
# 2. Haversine Mesafesi
#############################
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Dünya yarıçapı (km)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = (torch.sin(dlat / 2))**2 + torch.cos(lat1) * torch.cos(lat2) * (torch.sin(dlon / 2))**2
    c = 2 * torch.arcsin(torch.sqrt(a))
    return R * c


def normalize_latlon(lat, lon):
    """
    Latitude ve longitude değerlerini normalleştirir.
    """
    lat_norm = (lat - LAT_MEAN) / LAT_STD
    lon_norm = (lon - LON_MEAN) / LON_STD
    return lat_norm, lon_norm


def denormalize_latlon(lat_norm, lon_norm):
    """
    Normalleştirilmiş latitude ve longitude değerlerini gerçek değerlere çevirir.
    """
    lat = lat_norm * LAT_STD + LAT_MEAN
    lon = lon_norm * LON_STD + LON_MEAN
    return lat, lon


#############################
# 3. Veri Seti Sınıfı
#############################
class GeoDataset(Dataset):
    def __init__(self, csv_path, root_dir, transform=None, sep=';'):
        self.data_df = pd.read_csv(csv_path, sep=sep)
        self.root_dir = root_dir
        self.transform = transform

        self.filenames = self.data_df["filename"].values
        self.lats = self.data_df["latitude"].fillna(0.0).values
        self.lons = self.data_df["longitude"].fillna(0.0).values

    def __len__(self):
        return len(self.data_df)

    def __getitem__(self, idx):
        img_name = self.filenames[idx]
        lat = self.lats[idx]
        lon = self.lons[idx]

        lat_norm, lon_norm = normalize_latlon(lat, lon)

        img_path = os.path.join(self.root_dir, img_name)
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        coords = torch.tensor([lat_norm, lon_norm], dtype=torch.float32)
        return image, coords, img_name


#############################
# 4. Transform ve Dataset
#############################
image_size = 224
transform = Compose([
    Resize((image_size, image_size)),
    RandomHorizontalFlip(),
    RandomRotation(10),
    RandomResizedCrop(image_size, scale=(0.8, 1.0)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])

train_csv_path = "/kaggle/input/datathonai/train.csv"
train_images_dir = "/kaggle/input/datathonai/train"

dataset = GeoDataset(
    csv_path=train_csv_path,
    root_dir=train_images_dir,
    transform=transform
)
print("Train+Val Dataset Size:", len(dataset))

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)


#############################
# 5. Test Dataset
#############################
test_csv_path = "/kaggle/input/datathonai/test.csv"
test_images_dir = "/kaggle/input/datathonai/test"

test_dataset = GeoDataset(
    csv_path=test_csv_path,
    root_dir=test_images_dir,
    transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


#############################
# 6. ConvNeXt Modeli
#############################
model = AutoModelForImageClassification.from_pretrained(
    "facebook/convnextv2-large-1k-224",
    num_labels=2,
    ignore_mismatched_sizes=True
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)
model.to(device)


#############################
# 7. Kayıp Fonksiyonu, Optimizasyon, Scheduler
#############################
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1, verbose=True)


#############################
# 8. Eğitim Döngüsü
#############################
train_losses = []
val_losses = []

for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
    model.train()
    running_loss = 0.0

    for images, coords, _ in tqdm(train_loader, desc="Training"):
        images = images.to(device)
        coords = coords.to(device)

        optimizer.zero_grad()
        outputs = model(pixel_values=images)
        logits = outputs.logits
        loss = criterion(logits, coords)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    train_loss = running_loss / len(train_loader)
    train_losses.append(train_loss)

    # Validation
    model.eval()
    val_running_loss = 0.0
    distances = []
    with torch.no_grad():
        for val_images, val_coords, _ in tqdm(val_loader, desc="Validation"):
            val_images = val_images.to(device)
            val_coords = val_coords.to(device)

            val_logits = model(pixel_values=val_images).logits
            val_loss = criterion(val_logits, val_coords)
            val_running_loss += val_loss.item()

            for i in range(val_logits.size(0)):
                pred_lat_norm, pred_lon_norm = val_logits[i]
                pred_lat, pred_lon = denormalize_latlon(pred_lat_norm, pred_lon_norm)
                true_lat, true_lon = denormalize_latlon(val_coords[i][0], val_coords[i][1])

                dist = haversine_distance(
                    torch.tensor([math.radians(pred_lat)]),
                    torch.tensor([math.radians(pred_lon)]),
                    torch.tensor([math.radians(true_lat)]),
                    torch.tensor([math.radians(true_lon)])
                )
                distances.append(dist.item())

    val_loss = val_running_loss / len(val_loader)
    val_losses.append(val_loss)

    mean_distance = sum(distances) / len(distances)
    print(f"Train Loss: {train_loss:.5f} | Val Loss: {val_loss:.5f} | Mean Distance: {mean_distance:.5f} km")

    scheduler.step(val_loss)

    # Test Tahminleri
    model.eval()
    test_preds = []
    test_filenames = []
    with torch.no_grad():
        for test_images, _, test_names in tqdm(test_loader, desc="Test Inference"):
            test_images = test_images.to(device)
            test_out = model(pixel_values=test_images).logits

            for i, fname in enumerate(test_names):
                pred_lat_norm, pred_lon_norm = test_out[i]
                pred_lat, pred_lon = denormalize_latlon(pred_lat_norm, pred_lon_norm)
                test_preds.append((pred_lat.item(), pred_lon.item()))
                test_filenames.append(fname)

    df_test_out = pd.DataFrame({
        "filename": test_filenames,
        "latitude": [p[0] for p in test_preds],
        "longitude": [p[1] for p in test_preds]
    })
    test_csv_path = f"test_predictions_epoch_{epoch+1}.csv"
    df_test_out.to_csv(test_csv_path, sep=';', index=False)

    model_save_path = f"convnextv2_epoch_{epoch+1}.pth"
    torch.save(model.state_dict(), model_save_path)

print("Eğitim tamamlandı!")


Train+Val Dataset Size: 4375


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/792M [00:00<?, ?B/s]

Some weights of ConvNextV2ForImageClassification were not initialized from the model checkpoint at facebook/convnextv2-large-1k-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 1536]) in the checkpoint and torch.Size([2, 1536]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Epoch 1/10


...


Epoch 10/10


Validation: 100%|██████████| 14/14 [01:35<00:00,  6.84s/it]


Train Loss: 0.00703 | Val Loss: 0.08163 | Mean Distance: 0.07857 km


Test Inference: 100%|██████████| 25/25 [02:49<00:00,  6.77s/it]


Eğitim tamamlandı!
